# Qulacs & openfermion & psi4 で VQE

必要なもの

- qulacs
- openfermion
- openfermion-psi4
- psi4
- scipy
- numpy

In [ ]:
import sys, os
import qulacs

from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.transforms import get_sparse_operator
from openfermion.hamiltonians import MolecularData
from openfermionpsi4 import run_psi4

from scipy.optimize import minimize
import numpy as np
import matplotlib.pyplot as plt

## ハミルトニアンを作る

In [ ]:
basis = "sto-3g"
multiplicity = 1
charge = 0
trotter_step = 1
distance  = 0.977
geometry = [["H", [0,0,0]],["H", [0,0,distance]]]
description  = "tmp"
molecule = MolecularData(geometry, basis, multiplicity, charge, description)
molecule = run_psi4(molecule,run_scf=1,run_fci=1)

n_qubit = molecule.n_qubits
n_electron = molecule.n_electrons

### ハミルトニアンを変換・表示する

In [ ]:
fermionic_hamiltonian = get_fermion_operator(molecule.get_molecular_hamiltonian())
jw_hamiltonian = jordan_wigner(fermionic_hamiltonian)

In [ ]:
print(jw_hamiltonian)

### ハミルトニアンを qulacs ハミルトニアンに変換する

In [ ]:
from qulacs import Observable
from qulacs.observable import create_observable_from_openfermion_text

In [ ]:
qulacs_hamiltonian = create_observable_from_openfermion_text(str(jw_hamiltonian))

## ansatz を構成する

In [ ]:
from qulacs import QuantumState, QuantumCircuit
from qulacs.gate import CZ, RY, RZ, merge

depth = n_qubit

In [ ]:
def he_ansatz_circuit(n_qubit, depth, theta_list):
    """he_ansatz_circuit
    Retunrs hardware efficient ansatz circuit.

    Args:
        n_qubit (:class:`int`):
            the number of qubit used (equivalent to the number of fermionic modes)
        depth (:class:`int`):
            depth of the circuit.
        theta_list (:class:`numpy.ndarray`):
            rotation angles.
    Returns:
        :class:`qulacs.QuantumCircuit`
    """
    circuit = QuantumCircuit(n_qubit)
    
    for d in range(depth):
        for i in range(n_qubit):
            circuit.add_gate(merge(RY(i, theta_list[2*i+2*n_qubit*d]), RZ(i, theta_list[2*i+1+2*n_qubit*d])))
        for i in range(n_qubit//2):
            circuit.add_gate(CZ(2*i, 2*i+1))
        for i in range(n_qubit//2-1):
            circuit.add_gate(CZ(2*i+1, 2*i+2))
    for i in range(n_qubit):
        circuit.add_gate(merge(RY(i, theta_list[2*i+2*n_qubit*depth]), RZ(i, theta_list[2*i+1+2*n_qubit*depth])))
    
    return circuit

## Hartree Fock エネルギーのチェック

In [ ]:
input_state = QuantumState(n_qubit)
input_state.set_computational_basis(int('0b'+'0'*(n_qubit - n_electron)+'1'*(n_electron),2))

In [ ]:
qulacs_hamiltonian.get_expectation_value(input_state)

In [ ]:
molecule.hf_energy

## VQE の cost function

In [ ]:
def cost(theta_list):
    input_state.set_computational_basis(int('0b'+'0'*(n_qubit - n_electron)+'1'*(n_electron),2))
    circuit = he_ansatz_circuit(n_qubit, depth, theta_list)
    circuit.update_quantum_state(input_state)
    return qulacs_hamiltonian.get_expectation_value(input_state)

## 初期パラメータセット

In [ ]:
init_theta_list = np.random.random(2*n_qubit*(depth+1))*1e-1
init_theta_list

## 最適化

In [ ]:
method = "BFGS"
options = {"disp": True, "maxiter": 50, "gtol": 1e-6}
opt = minimize(cost, init_theta_list, method = method)

In [ ]:
opt.x

In [ ]:
print(opt.fun, float(molecule.fci_energy))